## Preprocess and load our dataset
For preprocessing, convert all images in folders to array (224, 224, 3) and scale them. 70% of data is stored in train dataset array, 20% goes to validation dataset, and 10% goes to test dataset. Finally, convert all datas to float and one-hot-encode labels.   
For creating a CNN model, I refered to https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

In [1]:
# Import for loading and preprocessing

import numpy as np
import cv2
from PIL import Image
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from os import listdir

RGB color code for texts. 0, 255, 0 is lime

In [2]:
color = (0,255,0)

Create a classifier for face detections

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

In [4]:
# Preprocess_input normalizes input in scale of [-1, +1]. You must apply same normalization in prediction.
# Devide the array by 127.5 and substract by 1
# Ref: https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py (Line 45)

def preprocess_image(file_path):
    img = load_img(file_path, target_size=(224, 224)) 
    img = img_to_array(img)
    img = preprocess_input(img)
    
    return img

In [5]:
# Initialize a file path, class name as int, and lists/arrays for dataset
pictures_folder = "New Faces"

our_class = []
class_name = 0

train_data = []
train_label = []
valid_data = []
valid_label = []

In [6]:
# Iterate a folder ("New Faces") to folder ("User name") to user's 150 face images
# String beocomes file path by listdir
for folder_name in listdir(pictures_folder):
    if folder_name != '.DS_Store':  # for mac users
        our_class.append(folder_name)
        
        folder_path = pictures_folder+'/'+folder_name  # create a path for the next folder to iterate
        
        index = 0  # reset index
        
        for file_name in listdir(folder_path):
            file_name = folder_path+'/'+file_name  # create a path for the files
            face_img = preprocess_image(file_name)  # scale and load images by the path
            
            size = len(listdir(folder_path))
            
            # 70% of data for training
            if index < size * 0.7: 
                train_data.append(face_img)
                train_label.append(class_name)
            
            # 30% of data for validation
            else:
                valid_data.append(face_img)
                valid_label.append(class_name)
            
            index += 1  # increment index
        
        class_name += 1

In [7]:
# Convert the list to numpy arrays of float
train_data = np.array(train_data).astype('float32')
valid_data = np.array(valid_data).astype('float32')

# One-hot-encode labels for 'crosscategorical_entropy' loss function
train_label = to_categorical(train_label)
valid_label = to_categorical(valid_label)

#train_data.shape

## Load pre-trained model
Import VGG16 from keras-applications and load the model (CNN model).   
Input shape is (224, 224, 3) that is size of images in our dataset.   
Please refer to https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a

In [8]:
import keras
from keras.applications import vgg16
from keras.models import Model
from keras.layers import Flatten

# The size of images in our dataset
input_shape = (224, 224, 3)

# Load VGG16 model
vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)
# Build the model
output = vgg.layers[-1].output
output = Flatten()(output)
vgg_model = Model(vgg.input, output)

# Freeze all layers in the base model by setting trainable = False
vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False

2022-05-05 08:57:09.636894: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Build a model with the pre-trained model
The layered architecture of deep learning model allows us to utilize a pre-trained network such as VGG16 without the final layer as a fixed feature extractor for the new task.   
For feature extracting, add VGG model as layers and other layers for output to a sequentail model.   
   
Loss is 0, and accuracy is 1

In [9]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

model = Sequential()
model.add(vgg_model)  # the pre-trained model
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(our_class), activation='softmax'))  # output layer

# Configure the learning process via the compilation method
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

# Display the model              
model.summary()             

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 25088)             14714688  
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 27,824,451
Trainable params: 13,109,763
No

In [10]:
from keras.callbacks import EarlyStopping

# Initialize the callback and add it to a list
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

callback_list = [es]

In [11]:
# Fit the models and apply the callback with its loss and the accuracy of each epoch
history = model.fit(train_data, train_label, batch_size=5, epochs=20, 
                    callbacks=callback_list, validation_data=(valid_data, valid_label), shuffle=True)

Epoch 1/20
63/63 [==============================] - 56s 887ms/step - loss: 2.4978 - accuracy: 0.9397 - val_loss: 0.6393 - val_accuracy: 0.9926
Epoch 2/20
63/63 [==============================] - 57s 906ms/step - loss: 1.3122 - accuracy: 0.9873 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
63/63 [==============================] - 58s 923ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
63/63 [==============================] - 58s 925ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
63/63 [==============================] - 56s 891ms/step - loss: 0.1880 - accuracy: 0.9937 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [13]:
# Saved in the same folder. It can be loaded anytime by load_model(path_name) function
model.save('fe_model' + '.h5')

In [14]:
# Load the saved model
from tensorflow import keras
our_model = keras.models.load_model('fe_model.h5')

In [15]:
# Calculate a cosine distance between two values. Distance between similar vectors should be low.

def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

## Predict on test data

In [16]:
img = preprocess_image("../jackman.png")
img = np.expand_dims(img, axis=0)
predict_array = our_model.predict(img)[0,:]


predict_array


array([0., 1., 0.], dtype=float32)

In [10]:
# Open webcam
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()  # get a image from webcam capture
    faces = face_cascade.detectMultiScale(img, 1.3, 5)  # find faces in images

    for (x,y,w,h) in faces:  # x, y represents initial posions in a graph. w is width and h is height
        if w > 130: 

            detected_face = img[int(y):int(y+h), int(x):int(x+w)] # crop detected face
            detected_face = cv2.resize(detected_face, (224, 224)) # resize to 224x224
            detected_face = img_to_array(detected_face)
            detected_face = np.expand_dims(detected_face, axis=0)
            
            
            img_pixels = preprocess_input(detected_face)
            
            # Assign the image extracted from webcam
            predict_array = our_model.predict(img_pixels)[0,:]
            
            pred = np.argmax(predict_array)
        

            found = 0  # counter
                
                    
                    # If cosine distance is small, consider it is k (user's name)
            if predict_array[pred] == 1.00: 
                cv2.putText(img, our_class[pred], (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                found = 1
                break

            # Connect face and text
            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)
            
            # if found image is not in user's face dataset, display unknown
            if(found == 0): 
                cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow('img',img)

    if cv2.waitKey(1) & 0xFF == ord('q'): # press q to quit
        break

# Kill open cv things
cap.release()
cv2.destroyAllWindows()
            

KeyboardInterrupt: 